In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

#------------------------------

import matplotlib.pyplot as plt

myparams = {
    'text.usetex': True,
    'text.latex.preamble': '\\usepackage{amsfonts}',
    'text.latex.preamble': '\\usepackage[utf8]{inputenc}',
    'text.latex.preamble': '\\usepackage[russian]{babel}',
    'font.family': 'Djvu Serif',
    'font.size': 14,
    'axes.grid': False,
    'grid.alpha': 0.1,
    'lines.linewidth': 2
}

plt.rcParams.update(myparams)

#------------------------------

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
def extract_last_word_from_filename(filename):
    # Используем регулярное выражение для поиска последнего слова в имени файла
    match = re.search(r'([^_]+)\.tsv$', filename)
    if match:
        return match.group(1)
    else:
        return None

In [20]:
filenames = []
objectnames = []

for root, dirs, files in os.walk('annotations/video'):
    for filename in files:
        filenames.append(os.path.join(root, filename))
        objectname = extract_last_word_from_filename(filename)
        objectnames.append(objectname)
        
filenames.sort()
objectnames.sort()

In [21]:
occur = list()
objects = dict()
pairs = list()

for filename, objectname in zip(filenames, objectnames):
    df = pd.read_csv(filename, sep='\t')
    df = (df * 25).astype(dtype=int)
    vector = np.zeros(9750, dtype=int)
    for ts in df.values:
        vector[ts[0]:ts[1]+1] = 1
    occur.append(vector)
    objects[objectname] = {}
    objects[objectname]['count'] = df.shape[0]
    objects[objectname]['occurences'] = vector
    pairs.append((objectname, df.shape[0]))
    
occur = np.array(occur)
pairs.sort(key=lambda x: x[1], reverse=True) # пары (объек, число появлений в кадре), отсортированные по убыванию числа появлений

Посмотрим, какой объект чаще всего появляется в кадре.

In [22]:
print(f"Object '{pairs[0][0]}' occured maximum number of times: {pairs[0][1]}")

Object 'pippi' occured maximum number of times: 26


Каждая строка `occur` соответствует одному объекту.

Каждый столбец `occur` соответствует одному кадру.

In [23]:
occur

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]])

Поисследуем немного полученные данные.

In [24]:
occur.sum(axis=0)

array([50, 50, 50, ..., 15, 15, 15])

In [25]:
len(objectnames)

135

In [26]:
objectnames.index('horse')

72